In [1]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import pi
import pickle as pkl
from qutip import *

In [2]:
%matplotlib qt5

In [3]:
f1 = open('Sample 1808-072_2 Coil #NNDAC-6 two-tone qubit2.pkl', 'rb')
data = pkl.load(f1)  

In [4]:
f = open('Sample 1808-072_2 Coil #NNDAC-6 two-tone qubit1.pkl', 'rb')
data1 = pkl.load(f)

In [5]:
flux = data[1]["S-parameter"][1][0]
freq = data[1]["S-parameter"][1][1]/1e9
S21 = data[1]["S-parameter"][2]
plt.figure(1)
plt.imshow((np.imag(S21)-np.mean(np.imag(S21))).T,cmap='RdBu',
                  extent=[np.min(flux),np.max(flux),np.min(freq),np.max(freq)],
                   aspect='auto',origin='lower')
plt.show()

In [6]:
flux1 = data1[1]["S-parameter"][1][0]
freq1 = data1[1]["S-parameter"][1][1]/1e9
S211 = data1[1]["S-parameter"][2]
plt.figure(2)
plt.imshow((np.imag(S211)-np.mean(np.imag(S211))).T,cmap='RdBu',
                  extent=[np.min(flux1),np.max(flux1),np.min(freq1),np.max(freq1)],
                   aspect='auto',origin='lower')
plt.show()

In [7]:
alpha= 0.175
wq1_max=5.887
wq1_min=4.987
wq2_max=5.915
wq2_min=4.986
w_r=6.7
g=0.040
bias = 0

In [19]:
# def qubit_frequency(flux,Ej1,Ej2,Ec,bias=0): # if flux in flux quanta
#     Ej=np.sqrt(((Ej1+Ej2)*np.cos(np.pi*flux+bias))**2+((Ej1-Ej2)*np.sin(np.pi*flux+bias))**2)
#     return np.sqrt(8*Ec*Ej)-Ec

In [19]:
def E_J(flux,Ej1,Ej2,bias=0): # if flux in flux quanta
    Ej=np.sqrt(((Ej1+Ej2)*np.cos(np.pi*flux+bias))**2+((Ej1-Ej2)*np.sin(np.pi*flux+bias))**2)
    return Ej

In [20]:
E_J(0,Ej11,Ej12)

26.248459999999998

In [21]:
def Ejs(minf,maxf,alpha):
    Ejmin=(minf+alpha)**2/(8*alpha)
    Ejmax=(maxf+alpha)**2/(8*alpha)
    Ej1 = (Ejmax-Ejmin)/2
    Ej2 = (Ejmax+Ejmin)/2
    return Ej1,Ej2  

In [22]:
Ej11,Ej12=Ejs(wq1_min,wq1_max,alpha)
Ej22,Ej21=Ejs(wq2_min,wq2_max,alpha)
Ej11,Ej12,Ej22,Ej21

(3.607714285714284, 22.640745714285714, 3.7329210714285708, 22.758578928571428)

In [23]:
import numpy as np

In [24]:
def n():
    return Qobj((np.diag(np.arange(-N_q, N_q+1, 1))))
def H_tr(E_C, curr, Ej1, Ej2,N_q):
    return 4*E_C*n()**2+ E_J(curr, Ej1, Ej2)/2*(Qobj(np.diag(np.ones(2*N_q), 1) + np.diag(np.ones(2*N_q), -1)))

In [40]:
E_C=0.175# это надо правильно подбирать

In [36]:
# E_C=1.106424911858447

## two-levels approximation

In [27]:
flux_points = 256+1
spectral_lines = np.zeros((2, flux_points))
wq_flux_dep = np.zeros((flux_points))
fluxes = np.linspace(0, 0.5, flux_points) #in flux quanta
for flux_id, flux in enumerate(fluxes):
    wq_flux1 = qubit_frequency(flux,Ej11,Ej12,alpha)
    wq_flux2 = qubit_frequency(max(fluxes)-flux,Ej22,Ej21,alpha)
    H =wq_flux2*tensor(sigmaz(), identity(2))/2 + wq_flux1*tensor(identity(2),sigmaz())/2+g* tensor(sigmax(), sigmax())
    energies, eigenvectors = H.eigenstates()
    ground_state_energy = energies[0]
    spectral_lines[:, flux_id] = np.sort(energies[1:3]-ground_state_energy)
plt.plot(fluxes, spectral_lines[0],":",color='green',linewidth=3)
plt.plot(fluxes, spectral_lines[1],":",color='green', linewidth=3)

#plt.plot(fluxes, (spectral_lines[1]+spectral_lines[0])/2,":",color='green', linewidth=3)
plt.show()


NameError: name 'qubit_frequency' is not defined

In [60]:
a=S211+S21
plt.imshow((np.imag(a)-np.mean(np.imag(a))).T,cmap='RdBu',
                  extent=[np.min(flux1),np.max(flux1),np.min(freq1),np.max(freq1)],
                   aspect='auto',origin='lower')
plt.colorbar()
plt.ylabel("Частота,ГГц")
plt.xlabel("Поток,$\Phi_0$")
plt.show()

## many levels

In [29]:
from tqdm import tqdm

In [44]:
curs = np.linspace(0, 0.5, 101)
N_q=7
g=0.025
gqr=0.06
evals_list = []
for cur in tqdm(curs):
#     H = tensor(H_tr(E_C, cur, Ej11,Ej12,N_q), qutip.identity(15))+\
#     tensor(qutip.identity(15), H_tr(E_C, max(curs)-cur, Ej22,Ej21,N_q))+g*tensor(n(), n())
    H = tensor(H_tr(E_C, cur, Ej11,Ej12,N_q), qutip.identity(15), qutip.identity(7))+\
       tensor(qutip.identity(15), H_tr(E_C, max(curs)-cur, Ej22,Ej21,N_q), qutip.identity(7))+\
       tensor(qutip.identity(15), qutip.identity(15), w_r*destroy(7)*create(7))+\
       g*tensor(n(), n(), qutip.identity(7))+\
        gqr*tensor(qutip.identity(15), n(), destroy(7)+create(7))+\
        gqr*tensor(n(), qutip.identity(15), destroy(7)+create(7))
    #H=H_tr(E_C, cur, Ej11,Ej12,N_q)
    evals_list.append(H.eigenenergies())
evals_arr = np.array(evals_list)

100%|██████████| 101/101 [02:49<00:00,  1.49s/it]


In [43]:
plt.plot(curs, evals_arr[:,1] - evals_arr[:,0], ":", color = "green",linewidth=3)
plt.show()

In [31]:
plt.plot(curs, evals_arr[:,1] - evals_arr[:,0], ":", color = "green",linewidth=3)
plt.plot(curs, evals_arr[:,2] - evals_arr[:,0], ":", color = "black",linewidth=3)
plt.plot(curs, (evals_arr[:,3] - evals_arr[:,0]), ":", color = "black")
plt.plot(curs, (evals_arr[:,4] - evals_arr[:,0]), ":", color = "grey")
plt.plot(curs, (evals_arr[:,4] - evals_arr[:,0]), ":", color = "grey")
plt.plot(curs/1, (evals_arr[:,6] - evals_arr[:,0])/2, ":", color = "red")
plt.plot(curs/1, (evals_arr[:,7] - evals_arr[:,0])/2, ":", color = "red")
plt.plot(curs/1, (evals_arr[:,8] - evals_arr[:,0])/2, ":", color = "red")
plt.plot(curs/1, (evals_arr[:,9] - evals_arr[:,0])/2, ":", color = "red")
plt.plot(curs/1, (evals_arr[:,9] - evals_arr[:,0])/2, ":", color = "red")
plt.plot(curs/1, (evals_arr[:,20] - evals_arr[:,0])/
         3, ":", color = "red")
plt.plot(curs/1, (evals_arr[:,21] - evals_arr[:,0])/3, ":", color = "red")
plt.show()

In [61]:
plt.plot(curs, evals_arr[:,1] - evals_arr[:,0], ":", color = "black",linewidth=3)
plt.plot(curs, evals_arr[:,2] - evals_arr[:,0], ":", color = "black",linewidth=3)
plt.plot(curs/1, (evals_arr[:,4] - evals_arr[:,0])/2, ":", color = "black",linewidth=3)
plt.plot(curs/1, (evals_arr[:,5] - evals_arr[:,0])/2, ":", color = "black",linewidth=3)
# plt.plot(curs/1, (evals_arr[:,6] - evals_arr[:,0])/2, ":", color = "black",linewidth=3)
# plt.plot(curs/1, (evals_arr[:,4] - evals_arr[:,1]), ":", color = "black",linewidth=3) #2photon1qubit
plt.show()

In [58]:
(evals_arr[:,1] - evals_arr[:,0])[0],\
(evals_arr[:,2] - evals_arr[:,0])[0],\
(evals_arr[:,4] - evals_arr[:,0])[0]/2,\
(evals_arr[:,5] - evals_arr[:,0])[0]/2,\
(evals_arr[:,6] - evals_arr[:,0])[0]/2

(4.975065893297078,
 5.881781733239237,
 4.880350765272931,
 5.42743853760841,
 5.789598950338725)

In [59]:
(4.975065893297078+5.881781733239237)/2

5.428423813268157

## with resonator

In [26]:
Nr=2
g_q_r = 0.06
w_r=6.6
a = destroy(Nr)

In [33]:
flux_points = 512+1
spectral_lines = np.zeros((3, flux_points))
wq_flux_dep = np.zeros((flux_points))
fluxes = np.linspace(0, 0.5, flux_points) #in flux quanta
for flux_id, flux in enumerate(fluxes):
    wq_flux1 = qubit_frequency1(flux,Ej11,Ej12,alpha)
    wq_flux2 = qubit_frequency1(max(fluxes)-flux,Ej22,Ej21,alpha)
    H =wq_flux2*tensor(sigmaz(), identity(2),identity(Nr))/2 +\
    wq_flux1*tensor(identity(2),sigmaz(),identity(Nr))/2 +\
    g* tensor(sigmax(), sigmax(),identity(Nr))+\
    g_q_r*tensor(sigmax(),identity(2),a+a.dag())+\
    g_q_r*tensor(identity(2),sigmax(),a+a.dag())+\
    w_r*tensor(identity(2),identity(2),a.dag()*a)
    energies, eigenvectors = H.eigenstates()
    ground_state_energy = energies[0]
    spectral_lines[:, flux_id] = np.sort(energies[1:4]-ground_state_energy)#,energies[4]-ground_state_energy])
plt.plot(fluxes, spectral_lines[0],":",color='green',linewidth=3)
plt.plot(fluxes, spectral_lines[1],":",color='green', linewidth=3)
plt.plot(fluxes, spectral_lines[2],"-",color='green', linewidth=3)
#plt.plot(fluxes, (spectral_lines[1]+spectral_lines[0])/2,":",color='green', linewidth=3)
plt.show()

NameError: name 'qubit_frequency1' is not defined

In [12]:
for i,x in enumerate(energies):
    for j,y in enumerate(energies):
        if(j>i):
            print(abs(x-y))

4.983725987443282
5.9115207143762145
6.608218043316809
10.895660351005485
11.59108337699763
12.52021126676722
17.503490249634062
0.9277947269329321
1.6244920558735263
5.911934363562203
6.607357389554347
7.536485279323937
12.519764262190781
0.6966973289405942
4.984139636629271
5.679562662621415
6.608690552391005
11.59196953525785
4.287442307688677
4.982865333680821
5.911993223450411
10.895272206317255
0.6954230259921443
1.6245509157617342
6.607829898628578
0.9291278897695898
5.912406872636434
4.983278982866844
